In [ ]:
import json
import numpy as np
import pandas as pd
import os

from math import cos, sin, exp, sqrt, pi, e

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset

## Constants to be used across the notebook

In [ ]:
# Min and Max temperatures to be received by the user.
# Values [40,...,60]
Tmin_i = float(input("Enter min setpoint temperature: "))
Tmax_i = float(input("Enter max setpoint temperature: "))

# Domestic hot water temperature.
# Values [45,...,55]
Tdhw = float(input("Enter domestic hot water temperature: "))

# Our model supposes that the system has two different sources of heat.
# Values [20,...,30]
Ts1 = float(input("Temperature of source 1: "))
Ts2 = float(input("Temperature of source 2: "))

# difference in temperature between the supply (system->heat->house) and return (house -> heat -> system) pipes
# Values [1,...,10]
DT_evap = float(input("Temperature difference among the supply and return pipes: "))

## Load Sources working hours

The file must be a boolean matrix $W$ of $24 x 7$. The rows represent the hours in a day, and the columns represent the days in the week. For all $i \in \{0,...,23\}$ and $j \in \{0,..., 6\}$ we have that $w_{i,j} \in \{0, 1\}$, where $w_{i,j} = 0$ means that the source doesn't produce energy at that hour $i$ on that day $j$. Similarly, $w_{i,j} = 1$ means that the source produces energy at that hour $i$ on that day $j$.

In [ ]:
# Time repr
s1_schedule_path = os.path.join(".", "data", "private", "s1_source_fake_schedule.xlsx")

s1_schedule = pd.read_excel(s1_schedule_path, index_col="Time").to_numpy()

print(f"* {s1_schedule_path=}\n"
      f"* {s1_schedule=}\n"
      f"* {s1_schedule[6,1]=}\n" # Tuesdays are closed at 6 (equal to 0)
      f"* {s1_schedule[7,1]=}\n") # but open are 7 (equal to 1)

In [ ]:
s2_schedule_path = os.path.join(".", "data", "private", "s2_source_fake_schedule.xlsx")
s2_schedule = pd.read_excel(s2_schedule_path, index_col="Time").to_numpy()

print(f"* {s2_schedule_path=}\n"
      f"* {s2_schedule=}\n"
      f"* {s2_schedule[15,1]=}\n" # Tuesdays are closed at 15 (equal to 0)
      f"* {s2_schedule[16,1]=}\n") # but open are 16 (equal to 1)

## Take note of the values for the current simulation

In [ ]:
simulation_values = dict()
simulation_values["Tmin_i"] = Tmin_i
simulation_values["Tmax_i"] = Tmax_i
simulation_values["Tdhw"] = Tdhw
simulation_values["Ts1"] = Ts1
simulation_values["Ts2"] = Ts2
simulation_values["DT_evap"] = DT_evap
simulation_values["s1_schedule"] = s1_schedule.tolist()
simulation_values["s2_schedule"] = s2_schedule.tolist()

## Read the dataset

In [ ]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.keys()

In [ ]:
dataset_to_work = "rochester_newyork"
dataset_to_work

In [ ]:
Tamb_hourly = AVAILABLE_DATASETS[dataset_to_work]
Tamb_hourly

## Calculate Space Heating and Hot Water Temperature

The space Heating is calculated using a climatic curve. The Hot Water temperature is constant across the year.

In [ ]:
def climatic_curve(Tamb_h):
    Tmin_o = 2.38 #minimum outdoor T threshold in which the space heating system turns on
    Tmax_o = 7.25 #maximum outdoor T threshold in which the space heating system turns off

    if Tamb_h <= Tmin_o:   
        Tsh = Tmax_i
    elif Tamb_h >= Tmax_o:
        Tsh = Tmin_i
    else:
        m = (Tmax_i-Tmin_i)/(Tmin_o-Tmax_o)
        b = -m*Tmin_o+Tmax_i
        Tsh = m*Tamb_h+b
    return Tsh
 

In [ ]:
Tamb_hourly['space_heating_temp'] = Tamb_hourly.air_temp.apply(climatic_curve)
Tamb_hourly["hot_water_temp"] = Tdhw
Tamb_hourly

## Calculate User Temperature

This is the temperature that the users receive by the system. The temperature is based on their personal preferences, and the current season of the year.

In [ ]:
def Tuser(fila):
    Tsh = fila.space_heating_temp
    Tdhw = fila.hot_water_temp
    month = fila.month
    
    #Las proporciones dependen de la localización
    if month==1:
        resultado= 0.98*Tsh+0.02*Tdhw
       
    elif month==2:
        resultado=0.96*Tsh+0.04*Tdhw
    
    elif month==3:
        resultado=0.94*Tsh+0.06*Tdhw
    
    elif month==4:
        resultado=0.90*Tsh+0.1*Tdhw
    
    elif month==5:
        resultado=0.75*Tsh+0.25*Tdhw
    
    elif month==6:
        resultado=0.25*Tsh+0.75*Tdhw
    
    elif month==7:
        resultado=0*Tsh+1.0*Tdhw
    
    elif month==8:
        resultado=0*Tsh+1.0*Tdhw
    
    elif month==9:
        resultado=0.1*Tsh+0.9*Tdhw
    
    elif month==10:
        resultado=0.80*Tsh+0.2*Tdhw
    
    elif month==11:
        resultado=0.95*Tsh+0.05*Tdhw
    
    elif month==12:
        resultado=0.97*Tsh+0.03*Tdhw
        
    return resultado

In [ ]:
Tamb_hourly["user_temp"] = Tamb_hourly.apply(Tuser, axis=1)
Tamb_hourly

## Calculate Sources Temperature based on Working Hours

In [ ]:
dayofweeks = Tamb_hourly.index.dayofweek
hours = Tamb_hourly.index.hour

In [ ]:
Tamb_hourly["source1_temp"] = Ts1 * s1_schedule[hours, dayofweeks]
Tamb_hourly["source2_temp"] = Ts2 * s2_schedule[hours, dayofweeks]
Tamb_hourly

## Calculate Network temperature

In [ ]:
def calculate_tnet(temp_s1, temp_s2, temp_aq):
    if temp_s1 == 0.0 and temp_s2 == 0.0:
        return temp_aq
    elif temp_s1 == 0.0:
        return temp_s2
    elif temp_s2 == 0.0:
        return temp_s1
    else:
        return np.mean([temp_s1, temp_s2])

In [ ]:
Tamb_hourly["net_temp"] = Tamb_hourly.apply(lambda fila: calculate_tnet(fila.source1_temp, fila.source2_temp, fila.aquifer_temp), axis=1)
Tamb_hourly

# Coefficient of performance (COP)

In [ ]:
DT_hx = 2.5
n_HP = 0.53
Te_o = Tamb_hourly.net_temp - DT_evap - DT_hx
Tc_o = Tamb_hourly.user_temp

Tamb_hourly['COP']= 1-n_HP + n_HP * (Tc_o + 273.15) / (Tc_o + DT_hx - Te_o)
Tamb_hourly

# Heat losses

In [ ]:
def heat_losses(T_net,T_gr, DT_evap):
    U = 2
    T_ret = T_net - DT_evap
    HL_s = (T_net - T_gr) * U / 1000 # Heat losses supply pipe [MWh]
    HL_r = (T_ret - T_gr) * U / 1000 # Heat losses return pipe [MWh]
    
    return pd.Series([HL_s, HL_r, HL_s + HL_r] , index=['E_loss_s','E_loss_r','E_loss_tot'])


In [ ]:
x = Tamb_hourly.apply(lambda fila: heat_losses(fila["net_temp"], fila["ground_temp"], DT_evap), axis=1, result_type='expand')

In [ ]:
Tamb_hourly['E_loss_s'] = x['E_loss_s']
Tamb_hourly['E_loss_r'] = x['E_loss_r']
Tamb_hourly['E_loss_tot'] = x['E_loss_tot']
Tamb_hourly

## Save CSV version of the dataset and simulation values

In [ ]:
AVAILABLE_DATASET_PATHS = {OspitalettoDataset.OSPITALETTO: ospitaletto2019.processed_dataset_path,
                           NOAA2010Dataset.MIAMI_FL: noaa2010Dataset.processed_miami_fl_dataset_path, 
                           NOAA2010Dataset.FRESNO_CA: noaa2010Dataset.processed_fresno_ca_dataset_path, 
                           NOAA2010Dataset.OLYMPIA_WA: noaa2010Dataset.processed_olympia_wa_dataset_path,
                           NOAA2010Dataset.ROCHESTER_NY: noaa2010Dataset.processed_rochester_ny_dataset_path}

Tamb_hourly.to_csv(path_or_buf=AVAILABLE_DATASET_PATHS[dataset_to_work])

In [ ]:
simulation_values_path = os.path.join(".", "data", "simulation_values.json")

with open(simulation_values_path, "w") as f:
    json.dump(simulation_values, f, sort_keys=True, indent=2)